# Alpaca Broker API Dashboard

### Set Logging Level

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

### Import Libraries

In [2]:
import ipywidgets as widgets
from IPython.display import clear_output

import datetime as dt

import pandas as pd
import json

from utils import filters
from utils import api_hooks as ah
from utils import nb_funcs as nb
import broker_app as ab

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


### Constants

In [ ]:
timeframe_scale_dict = {
    'Min': range(1, 60),
    'Hour': range(1, 24),
    'Day': [1],
    'Week': [1],
    'Month': range(1, 13) 
}

## Set Output Window

In [3]:
out = widgets.Output(layout={'border': '1px solid black'})

### Create Widgets

In [4]:
class_dropdown = widgets.Dropdown(
    options=[],
    description='Asset Class:',
    disabled=False,
)

In [5]:
exchange_dropdown = widgets.Dropdown(
    options=[],
    description='Exchange:',
    disabled=False,
)

In [6]:
asset_dropdown = widgets.Dropdown(
    options=[],
    description='Symbol:',
    disabled=False,
)

In [7]:
timeframe_scale_dropdown = widgets.Dropdown(options=[], description='Timeframe')

In [8]:
timeframe_unit_dropdown = widgets.Dropdown()

In [9]:
start_date_picker = widgets.DatePicker(description='Start')


In [10]:
timeframe_panel = widgets.HBox([timeframe_scale_dropdown, timeframe_unit_dropdown, start_date_picker])

In [11]:
asset_menu = widgets.HBox([class_dropdown, exchange_dropdown, asset_dropdown])

### Set Event Handlers

In [12]:
def set_exchanges(change):
    exchange_dropdown.options = filters.filter_exchanges(change['new'])

class_dropdown.observe(set_exchanges, names='value')

In [14]:
def set_assets(change):
    asset_dropdown.options = filters.filter_assets(change['new'])
    
exchange_dropdown.observe(set_assets, names='value')

In [18]:
def set_timeframe_scale(change):
    timeframe_scale_dropdown.options = timeframe_scale_dict[change['new']]

timeframe_unit_dropdown.observe(set_timeframe_scale, names='value')

### Set Widget Values

In [16]:
class_dropdown.options = ab.class_list
class_dropdown.value = 'us_equity'

In [ ]:
timeframe_unit_dropdown.options = timeframe_scale_dict.keys()

In [ ]:
start_date_picker.value = dt.datetime.now().date()

In [20]:
def show_asset_detail(asset, timeframe_scale, timeframe_unit):
    timeframe = f"{timeframe_scale}{timeframe_unit}"
    asset_name = filters.get_asset_name(asset)
    asset_details = filters.get_asset_by_symbol(asset)
    start_date = start_date_picker.value.isoformat()
    bars = ah.get_bars(asset, timeframe=timeframe, start=start_date)
    try:
        display(bars['next_page_token'])
        bars_df = pd.DataFrame(bars['bars'])
        bars_df.set_index('t')
    except Exception as e:
        display(e)
    else:
        display(asset_name, bars_df.head())


In [21]:
asset_detail_panel = widgets.interactive_output(
    show_asset_detail,
    {
        'asset': asset_dropdown,
        'timeframe_scale': timeframe_scale_dropdown,
        'timeframe_unit': timeframe_unit_dropdown,
    }
)

In [22]:
with out:
    display(asset_menu)
    display(timeframe_panel)
    display(asset_detail_panel)

In [23]:
out

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…